# <u>T16 HOI HUB</u>

## Import Libraries

In [1]:
from ipywidgets import widgets
from IPython.display import display, HTML
import os
import io
import base64
import json

## Data Exploration

In [2]:
video_folder = os.listdir('data/video')

list_of_video = {}
for video in video_folder:
    # remove hidden file especially for macOS
    if not video.startswith('.'):
        list_of_video[video] = video

## Create Caption files

In [3]:
# read action and frame object from file
def readCaptionFile(filename,videoName):
    # Opening JSON file
    f = open(filename)

    # returns JSON object as 
    # a dictionary
    data = json.load(f)

    # Iterating through the json
    # Closing file
    f.close()
    return data[videoName]["actions"]

# convert frame to time
def convertFrameToTime(frame):
    seconds = int(frame/25)
    minutes = "00"
    if seconds >= 60:
        minutes = str(seconds // 60)
        seconds = seconds % 60
    if len(minutes) == 1:
        minutes = "0" + minutes
    seconds = str(seconds)
    #may need handle hour
    if len(seconds) == 1:
        seconds = "0" + seconds 
    return (minutes + ":" + seconds + ".000")

# read reference text from txt file
def readReferenceFile(refFile):
    referenceDict = {}
    with open(refFile) as f:
        lines = f.readlines()
    for i in lines:
        x = i.split()
        referenceDict[str(x[0])] = x[1]
    return referenceDict

# create caption file
def formatCaptionFile(captionList, reference, captionPath):
    start = "WEBVTT\n\n"
    captions = []
    for i in captionList:
        text = reference[str(i[0])]
        lines = convertFrameToTime(i[1]) + " --> " + convertFrameToTime(i[2]) + "\n" + text + "\n\n"
        captions.append(lines)
    f = open(captionPath, "w")
    f.write(start)
    f.writelines(captions)
    f.close()

In [5]:
video_dropdown = widgets.Dropdown(
    options = list_of_video,
    description = 'Videos',
)

video_src = 'data/video/' + video_dropdown.value

def play_video(video_src,caption_src):
    video = io.open(video_src, 'r+b').read()
    encoded = base64.b64encode(video)
    return(HTML(data='''<video width="650" height="360" controls>
        <source src="data:video/mp4;base64,{0}" type="video/mp4" />
        <track kind="captions" src={1} srclang="en" label="English" default>
        </video>'''.format(encoded.decode('ascii'),caption_src)))


# video dropdown onchange function
def video_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        change['new']
        global video_src 
        video_src = 'data/video/'+ video_dropdown.value
        
          
# display video dropdown
video_dropdown.observe(video_on_change)
display(video_dropdown) 


Dropdown(description='Videos', options={'P02T01C06.mp4': 'P02T01C06.mp4', 'P02T01C07.mp4': 'P02T01C07.mp4'}, v…

In [6]:
# caption
videoName = video_dropdown.value.split(".")
#location of reference are place at root
ref = readReferenceFile('all_labels.txt')
# may need change the caption path to dynamic
captionPath = "data/video/" + videoName[0] + ".vtt"
# model result file should be some directory, here using root 
captionList = readCaptionFile('smarthome_CS_51.json',videoName[0])
formatCaptionFile(captionList,ref,captionPath)


video = video_src.split('/')[-1]
print("Currently playing : " + video)
play_video(video_src, captionPath)

FileNotFoundError: [Errno 2] No such file or directory: 'all_labels.txt'

## Inference

## Training HOI ML Model

## Evaluate

In [10]:
pretrained_model_folder = os.listdir('data/pretrained_model')

list_of_ptModels = {}
for model in pretrained_model_folder:
    # remove hidden file especially for macOS
    if not model.startswith('.'):
        list_of_ptModels[model] = model

evaluation_dropdown = widgets.Dropdown(
    options = list_of_ptModels,
    description = 'evaluation',
)

model_src = 'data/pretrained_model/' + evaluation_dropdown.value

# evaluation dropdown onchange function
def evaluation_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        change['new']
        global model_src 
        model_src = 'data/video/'+ evaluation_dropdown.value
        
          
# display evaluation dropdown
evaluation_dropdown.observe(evaluation_on_change)
display(evaluation_dropdown) 

Dropdown(description='evaluation', options={'PDAN_TSU_RGB': 'PDAN_TSU_RGB'}, value='PDAN_TSU_RGB')

In [3]:
###Evaluate Model
run = f""
!python test.py -dataset TSU -mode rgb -split_setting CS -model PDAN -train False -num_channel 512 -lr 0.0002 -kernelsize 3 -APtype map -epoch 140 -batch_size 1 -comp_info TSU_CS_RGB_PDAN -load_model $model_src


Traceback (most recent call last):
  File "C:\Users\ngjun\Downloads\ict3104-team16-2022\test.py", line 6, in <module>
    import torch
  File "C:\Users\ngjun\miniconda3\envs\ict3104\lib\site-packages\torch\__init__.py", line 124, in <module>
    raise err
OSError: [WinError 1455] The paging file is too small for this operation to complete. Error loading "C:\Users\ngjun\miniconda3\envs\ict3104\lib\site-packages\torch\lib\cudnn_adv_infer64_8.dll" or one of its dependencies.
